## **Problem 1: Generating Random Boolean Functions**


In [ ]:
# Problem 1

## **Problem 2: Classical Testing for Function Type**

In [ ]:
# Problem 2

## **Problem 3: Quantum Oracles**

In [ ]:
# Problem 3

## **Problem 4: Deutsch's Algorithm with Qiskit**

In [ ]:
# Problem 4

## **Problem 5: Scaling to the Deutsch–Jozsa Algorithm**

In [ ]:
# Problem 5